In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs

In [3]:
#get an url list 
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,17):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [25]:

def get_house_info(url, price):
    #url = 'https://www.airbnb.com/rooms/3503946?s=P8Cyps_7'
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
                
      
        if re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text) != None:
            accommodates = re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text).group(1)
        else:
            accommodates = re.search('"guest_label":"([0-9][0-9]?) guest.*', response.text).group(1)
        #print (accommodates)
        
        
        #bedrooms
        if re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text) != None:
            bedrooms = (re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text)).group(1)
        else:
            bedrooms = "studio"
            #print (bedrooms)
            
        #bathrooms
        
        try:
            bathrooms = (re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)).group(1)
        except AttributeError:
            bathrooms = (re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text))
        
        #beds
        beds = (re.search('"bed_label":"(.).*","bedroom_label"', response.text)).group(1)
        #print (beds)
        
        #room type
        try:
            room_type = results_page.find('span',class_="_1hh2h7tb").text
        except AttributeError:
            room_type='NoneType'

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [26]:
def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver  
    from selenium.webdriver.common.by import By
    browser = webdriver.Chrome('/Users/yutingzhang/Desktop/ops_con/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=100&price_max=199&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    #browser.quit()
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    prices = browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list = []
    for price in prices:
        try:
            price = price.text
            price_list.append(price)
        except:
            pass
    price_list = price_list[::3]
    browser.quit()
    print("length of price list is: ", len(price_list))

    #get the url for every apt on this page
    results_page = BeautifulSoup(html_source, "lxml")
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    #urls = browser.find_elements(By.XPATH,"//a[@class='_1ol0z3h']")
    url_list = []
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print("length of apt_url_list is:" + str(len(url_list)))
    print("-----------------")
    #print(url_list)
    #pass url and price to the get_house_info function and extract the rest infomation 
    
    if (len(price_list) == len(url_list)) and (len(price_list)>0):
        for x in range(len(price_list)):
            #print(get_house_info(url_list[x], price_list[x]))
            info_list.append(get_house_info(url_list[x], price_list[x]))
        #print('info list is:')
        #print(info_list)
        df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 
                                            'bathrooms', 'room_type', 'number_of_reviews',  'rating', 
                                            'lat', 'lon', 'superhost', 'response_rate'])
        #print('dataframe is:')
        #print(df2)
        
        dataframe=pd.concat([dataframe,df2], ignore_index=True)
        print('complete dataframe')

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [7]:
page_url='https://www.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&place_id=ChIJYeZuBI9YwokRjMDs_IEyCwo&price_min=30&price_max=53&allow_override%5B%5D=&s_tag=yg0bIn4m'
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #create a df   
    df=get_prices_urls(url,df)

length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:0
-----------------
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  0
length of apt_url_list is:0
--------------

In [8]:
df

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$49 per night,2,1,1,1,Private room in apartment,554 Reviews,Rating 4.5 out of 5,40.82264063496292,-73.940408848972,false,100%
1,Price\n$51 per night,2,1,2,1,Private room in apartment,134 Reviews,Rating 4.5 out of 5,40.7915379886083,-73.94233066326686,false,90%
2,Price\n$47 per night,2,1,1,1,Private room in apartment,220 Reviews,Rating 4.5 out of 5,40.79956538782032,-73.96468013665819,false,100%
3,Price\n$53 per night,2,1,2,1,Private room in apartment,78 Reviews,Rating 4 out of 5,40.791416419842115,-73.94664105824592,false,90%
4,Price\n$33 per night,2,studio,1,1,Private room in apartment,33 Reviews,Rating 4 out of 5,40.783392824241574,-73.95168451916678,false,100%
5,Price\n$39 per night,5,1,3,2,Private room in house,125 Reviews,Rating 5 out of 5,40.7943906533419,-73.94602445950284,"true,",97%
6,Price\n$50 per night,6,1,3,1,Private room in apartment,183 Reviews,Rating 4.5 out of 5,40.790244340768666,-73.94063548348574,false,100%
7,Price\n$42 per night,3,1,1,1,Private room in apartment,86 Reviews,Rating 4.5 out of 5,40.71279986335466,-73.98908878318653,false,100%
8,Price\n$34 per night,4,1,2,1,Private room in apartment,262 Reviews,Rating 4.5 out of 5,40.810503632446604,-73.9250705679171,false,91%
9,Price\n$49 per night,4,1,2,1,Private room in apartment,100 Reviews,Rating 4.5 out of 5,40.793911791338026,-73.94431553857629,"true,",97%


In [9]:
df.to_csv('/Users/yutingzhang/Desktop/airbnbdata30-53.csv')

In [27]:
page_url='https://www.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&place_id=ChIJYeZuBI9YwokRjMDs_IEyCwo&price_min=54&price_max=75&allow_override%5B%5D=&s_tag=yg0bIn4m'
url_list=get_url_list(page_url)
df1 = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #create a df   
    df1=get_prices_urls(url,df1)

length of price list is:  18
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:0
-----------------
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:0
-----------------
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length

In [28]:
df1

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$64 per night,2,1,1,1,Private room in apartment,336 Reviews,Rating 4.5 out of 5,40.77632479255753,-73.95457505681182,false,NaN
1,Price\n$62 per night,2,1,1,1,Private room in apartment,206 Reviews,Rating 5 out of 5,40.80011761454228,-73.94087265925425,"true,",NaN
2,Price\n$70 per night,2,1,1,1,Private room in apartment,88 Reviews,Rating 4.5 out of 5,40.757034474484605,-73.99371155074375,false,NaN
3,Price\n$67 per night,2,1,1,1,Private room in apartment,198 Reviews,Rating 4.5 out of 5,40.74864210562691,-74.00225901166331,false,NaN
4,Price\n$70 per night,1,1,1,1,Private room in loft,105 Reviews,Rating 4.5 out of 5,40.79320766774083,-73.97535698015427,"true,",NaN
5,Price\n$60 per night,2,1,1,1,Private room in apartment,39 Reviews,Rating 5 out of 5,40.75825740551617,-73.968719788299,"true,",NaN
6,Price\n$70 per night,2,1,1,1,Private room in apartment,177 Reviews,Rating 4.5 out of 5,40.718153734253974,-73.98936924627012,false,NaN
7,Price\n$55 per night,2,1,1,1,Private room in apartment,23 Reviews,Rating 4.5 out of 5,40.71982535210372,-73.98770139498315,false,NaN
8,Price\n$70 per night,4,studio,3,1,Entire apartment,66 Reviews,Rating 4.5 out of 5,40.75240830535957,-73.97121128877214,false,NaN
9,Price\n$65 per night,2,1,1,1,Private room in apartment,3 Reviews,Rating 5 out of 5,40.797642233434274,-73.95607259207551,false,NaN


In [ ]:
df1.to_csv('airbnbdata30-53.csv', header=False, mode = 'a')

In [31]:
page_url='https://www.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&place_id=ChIJYeZuBI9YwokRjMDs_IEyCwo&price_min=76&price_max=90&allow_override%5B%5D=&s_tag=yg0bIn4m'
url_list=get_url_list(page_url)
df2 = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #create a df   
    df2=get_prices_urls(url,df2)

length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:0
-----------------
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataf

In [32]:
df2

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$83 per night,4,1,2,1,Private room in apartment,97 Reviews,Rating 4.5 out of 5,40.722234641605674,-73.99526741088673,false,98%
1,Price\n$90 per night,3,1,3,1,NoneType,NoneType,no rating,40.808426509307374,-73.9412452797624,"true,",no info
2,Price\n$90 per night,3,1,1,1,Entire apartment,218 Reviews,Rating 4.5 out of 5,40.76981088248375,-73.95812213521074,false,100%
3,Price\n$80 per night,4,studio,1,1,Entire apartment,136 Reviews,Rating 4.5 out of 5,40.77091628159295,-73.94882262604739,false,100%
4,Price\n$89 per night,4,studio,1,1,Entire apartment,121 Reviews,Rating 4.5 out of 5,40.7718146112811,-73.95145442581493,"true,",100%
5,Price\n$84 per night,3,studio,1,1,Entire apartment,45 Reviews,Rating 4.5 out of 5,40.71553042533876,-73.99117817625118,false,100%
6,Price\n$79 per night,3,studio,2,1,Entire apartment,65 Reviews,Rating 4 out of 5,40.73878992564724,-73.99771916899928,false,100%
7,Price\n$85 per night,2,1,1,1,Private room in apartment,330 Reviews,Rating 5 out of 5,40.76444403535851,-73.98964724399595,"true,",100%
8,Price\n$90 per night,2,1,1,1,Private room in apartment,41 Reviews,Rating 5 out of 5,40.75964291399547,-73.98730828754057,false,100%
9,Price\n$85 per night,3,1,1,1,Private room in apartment,161 Reviews,Rating 5 out of 5,40.76312587188399,-73.98977939912263,false,100%


In [33]:
df2.to_csv('airbnbdata30-53.csv', header=False, mode = 'a')

In [34]:
page_url='https://www.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&place_id=ChIJYeZuBI9YwokRjMDs_IEyCwo&price_min=91&price_max=120&allow_override%5B%5D=&s_tag=yg0bIn4m'
url_list=get_url_list(page_url)
df3 = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #create a df   
    df3=get_prices_urls(url,df3)

length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  16
length of apt_url_list is:0
-----------------
length of price list is:  0
length of apt_url_list is:0
-----------------
length of price list is:  16
length of apt_url_list is:16
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  18
length of apt_url_list is:18
-----------------
complete datafr

In [35]:
df3

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$110 per night,3,studio,1,1,Entire apartment,246 Reviews,Rating 4.5 out of 5,40.75802570505719,-73.99222887094635,false,NaN
1,Price\n$99 per night,5,2,3,2,Entire condominium,32 Reviews,Rating 5 out of 5,40.72067221191618,-74.00023559020492,"true,",NaN
2,Price\n$100 per night,3,studio,1,1,Entire house,225 Reviews,Rating 4.5 out of 5,40.82248476602039,-73.94891210677788,false,NaN
3,Price\n$109 per night,4,1,2,1,Entire apartment,224 Reviews,Rating 4.5 out of 5,40.759210015052595,-73.98895470236982,false,NaN
4,Price\n$110 per night,2,studio,1,1,Entire apartment,246 Reviews,Rating 4.5 out of 5,40.75752444102327,-73.99425405806588,false,NaN
5,Price\n$95 per night,1,1,1,1,Private room in apartment,269 Reviews,Rating 5 out of 5,40.764245247791884,-73.99162127230096,"true,",NaN
6,Price\n$119 per night,3,1,2,1,Entire apartment,143 Reviews,Rating 4.5 out of 5,40.75462449096644,-73.9686900560429,false,NaN
7,Price\n$109 per night,6,studio,3,1,Entire apartment,142 Reviews,Rating 4.5 out of 5,40.750326668007986,-73.98547495281177,false,NaN
8,Price\n$120 per night,3,1,1,1,NoneType,NoneType,no rating,40.73174521428503,-73.98755508025062,"true,",no info
9,Price\n$111 per night,4,1,2,1,Entire apartment,155 Reviews,Rating 4.5 out of 5,40.73745212749196,-74.00725143888869,false,97%


In [36]:
df3.to_csv('airbnbdata30-53.csv', header=False, mode = 'a')

In [48]:
data=pd.read_csv('airbnbdata30-53.csv')

In [52]:
data.reset_index (drop = True)

,Unnamed: 0,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,0,Price\n$49 per night,2,1,1,1.0,Private room in apartment,554 Reviews,Rating 4.5 out of 5,40.822641,-73.940409,false,100%
1,1,Price\n$51 per night,2,1,2,1.0,Private room in apartment,134 Reviews,Rating 4.5 out of 5,40.791538,-73.942331,false,90%
2,2,Price\n$47 per night,2,1,1,1.0,Private room in apartment,220 Reviews,Rating 4.5 out of 5,40.799565,-73.964680,false,100%
3,3,Price\n$53 per night,2,1,2,1.0,Private room in apartment,78 Reviews,Rating 4 out of 5,40.791416,-73.946641,false,90%
4,4,Price\n$33 per night,2,studio,1,1.0,Private room in apartment,33 Reviews,Rating 4 out of 5,40.783393,-73.951685,false,100%
5,5,Price\n$39 per night,5,1,3,2.0,Private room in house,125 Reviews,Rating 5 out of 5,40.794391,-73.946024,"true,",97%
6,6,Price\n$50 per night,6,1,3,1.0,Private room in apartment,183 Reviews,Rating 4.5 out of 5,40.790244,-73.940635,false,100%
7,7,Price\n$42 per night,3,1,1,1.0,Private room in apartment,86 Reviews,Rating 4.5 out of 5,40.712800,-73.989089,false,100%
8,8,Price\n$34 per night,4,1,2,1.0,Private room in apartment,262 Reviews,Rating 4.5 out of 5,40.810504,-73.925071,false,91%
9,9,Price\n$49 per night,4,1,2,1.0,Private room in apartment,100 Reviews,Rating 4.5 out of 5,40.793912,-73.944316,"true,",97%


In [53]:
data.to_csv('airbnbdata30-120final.csv')